### Redes de classificação
Selecione um dataset e crie uma rede de classificação. Avalie o resultado com duas configurações diferentes. As configurações devem mudar no mínimo 3 hiperparâmetros (por exemplo, parâmetros do optimizer,  loss, número de camadas, função de ativação, normalização).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torchvision.models import MobileNet_V3_Small_Weights
from torch.utils.data import DataLoader

In [ ]:
weights = MobileNet_v3_Small_Weights.DEFAULT

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=weights.transforms())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=weights.transforms())

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

NameError: name 'mobilenet_v3_small_Weights' is not defined

In [ ]:
model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)